# 1. Setup Azure
In this notebook, we will set up the project in Azure by creating the resources we need.

We start by creating the resource group that we'll put all resources into. Then we create our storage account which will be used to store all data (include logs). Finally we set up our Service Bus namespace, and a queue attached to it.

---

### Import packages and load .env

In [1]:
import re
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import os
import json

In [2]:
env_path = find_dotenv()
if env_path=='':
    Path('.env').touch()
    env_path = find_dotenv()
load_dotenv(env_path)

True

### Define variables

Set variables for the project:

- `subscription_id` - the subscription id for your Azure account. Use `az account list -o table` to list all subscriptions
- `resource_group` - the name for the resource group you'll be using for this project. You can think of resource groups as logical containers for the resources you'll create in this tutorial.
- `region` - the region you wish to deploy your resources in:
    - You can see a list of the regions under the key 'name' when running the command `az account list-locations`. 
    - Not all regions support GPU enabled VMs. You can check [here](https://azure.microsoft.com/en-us/pricing/details/virtual-machines/linux/)
    
Set your storage account variables

- `storage_account_name` - the desired name of your storage account
- `storage_container_name` - the desired container name in the storage account is a logical container for individual blobs. For this project, we'll store all content into a single container for ease-of-use
- `model_dir` - the desired name of directory you wish to store your models

Set the Service Bus variables:

- `namespace` - the desired namespace for your Service Bus - this can simply be thought of as a logical container for Service Bus
- `queue` - the desired name of the queue. The queue belongs to the namespace.

Set your docker container variables:
- `docker_login` - the login username for your docker account
- `aks_image_repo` - the desired image repo to use for your aks container
- `aci_image_repo` - the desired image repo to use for your aci container

AKS variables:
- `aks_cluster` - the desired name of the aks cluster

Logic App deployment:
- `logic_app` - the desired name of your logic apps
- `aci_group` - the desired name of your aci group
- `aci_display_name` - the display name used for your aci group

In [3]:
!az account list-locations

[
  {
    "displayName": "China North",
    "id": "/subscriptions/43f94930-ca31-4ddc-91c2-2f1be4f0e8f3/locations/chinanorth",
    "latitude": "39.9788",
    "longitude": "116.4959",
    "name": "chinanorth",
    "subscriptionId": null
  },
  {
    "displayName": "China East",
    "id": "/subscriptions/43f94930-ca31-4ddc-91c2-2f1be4f0e8f3/locations/chinaeast",
    "latitude": "31.3209",
    "longitude": "121.5891",
    "name": "chinaeast",
    "subscriptionId": null
  },
  {
    "displayName": "China North 2",
    "id": "/subscriptions/43f94930-ca31-4ddc-91c2-2f1be4f0e8f3/locations/chinanorth2",
    "latitude": "39.9788",
    "longitude": "116.4959",
    "name": "chinanorth2",
    "subscriptionId": null
  },
  {
    "displayName": "China East 2",
    "id": "/subscriptions/43f94930-ca31-4ddc-91c2-2f1be4f0e8f3/locations/chinaeast2",
    "latitude": "31.3209",
    "longitude": "121.5891",
    "name": "chinaeast2",
    "subscriptionId": null
  }
]


In [4]:
subscription_id = "43f94930-ca31-4ddc-91c2-2f1be4f0e8f3"   # fill in
resource_group = "cambridge"           # fill in
region = "chinaeast2"                    # fill in

storage_account_name = "cambridgesa"            # feel free to replace or use this default
storage_container_name = "cambridgesc"                       # feel free to replace or use this default
model_dir = "models"                                 # feel free to replace or use this default

namespace = "cambridgena"                # feel free to replace or use this default
queue = "cambridgequ"                        # feel free to replace or use this default

docker_login = "pjh177787"                      # fill in
scoring_image_repo = "cambridgesp"    # feel free to replace or use this default
flask_image_repo = "cambridgefp"        # feel free to replace or use this default

aks_cluster = "cambridgecl"                # feel free to replace or use this default
logic_app = "cambridgela"                       # feel free to replace or use this default

In [5]:
set_key(env_path, "SUBSCRIPTION_ID", subscription_id)
set_key(env_path, "RESOURCE_GROUP", resource_group)
set_key(env_path, "REGION", region)
set_key(env_path, "STORAGE_ACCOUNT_NAME", storage_account_name)
set_key(env_path, "STORAGE_CONTAINER_NAME", storage_container_name)
set_key(env_path, "STORAGE_MODEL_DIR", model_dir)
set_key(env_path, "SB_NAMESPACE", namespace)
set_key(env_path, "SB_QUEUE", queue)
set_key(env_path, "DOCKER_LOGIN", docker_login)
set_key(env_path, "SCORING_IMAGE", scoring_image_repo)
set_key(env_path, "FLASK_IMAGE", flask_image_repo)
set_key(env_path, "AKS_CLUSTER", aks_cluster)
set_key(env_path, "LOGIC_APP", logic_app)

(True, 'LOGIC_APP', 'cambridgela')

### Set up your resource group
Make sure you've identified which subscription_id and region to use. Create a new resource group to contain all the resources that we create.

This section of the notebook will walk through setting up the resource group using the __az cli__.

In [6]:
!az login -u zjdemo-02@ZJAIIoTLab.partner.onmschina.cn -p ZJintern002

[
  {
    "cloudName": "AzureChinaCloud",
    "id": "43f94930-ca31-4ddc-91c2-2f1be4f0e8f3",
    "isDefault": true,
    "name": "Windows Azure 企业",
    "state": "Enabled",
    "tenantId": "1c1fad78-2e51-4908-a7ee-d73cf1ea1aec",
    "user": {
      "name": "zjdemo-02@ZJAIIoTLab.partner.onmschina.cn",
      "type": "user"
    }
  }
]


In [7]:
!az account set -s {subscription_id}

Create the resource group which we'll put our storage account and all other resources for this project into.

In [8]:
!az group create -l {region} -n {resource_group}

{
  "id": "/subscriptions/43f94930-ca31-4ddc-91c2-2f1be4f0e8f3/resourceGroups/cambridge",
  "location": "chinaeast2",
  "managedBy": null,
  "name": "cambridge",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": null
}


### Create Azure blob storage
In this section of the notebook, we'll create an Azure blob storage that we'll use throughout the tutorial. This object store will be used to store input and output images as well as any supplementary data such as logs and other scripts that will be used in this workflow.

Use the __az cli__ to create the account

In [9]:
!az storage account create -n {storage_account_name} -g {resource_group} --query 'provisioningState'

An operation is currently performing on this storage account that requires exclusive access.


Use the __az cli__ to grab the keys of the storage account that was just created. The `--quote '[0].value'` part of the command simply means to select the _value_ of the _zero-th indexed_ of the set of keys.

In [35]:
key = !az storage account keys list --account-name {storage_account_name} -g {resource_group} --query '[0].value'

The stdout from the command above is stored in a string array of 1. Select the element in the array and ttrip opening and closing quotation marks.

In [36]:
storage_account_key = str(key[0][1:-1]) # this is used to strip opening and closing quotation marks

Use the __az cli__ to create the container in the storage account

In [37]:
!az storage container create \
    --account-name {storage_account_name} \
    --account-key {storage_account_key} \
    --name {storage_container_name}

{
  "created": true
}


Set storage account key to dotenv file.

In [38]:
set_key(env_path, "STORAGE_ACCOUNT_KEY", storage_account_key) # generated

(True,
 'STORAGE_ACCOUNT_KEY',
 '8JkanaXYAxmVdXoh9+Hbyr5EqTv5S4g+6ikHWt6JMZ2ivtSNKKWpjVy3vZALcSK4mQVRW3S+lVgoL6xFEHEIDw==')

### Create Service Bus & Generated Keys

Create the namespace.

In [39]:
!az servicebus namespace create \
    --resource-group {resource_group} \
    --name {namespace} 

{
  "createdAt": "2019-07-18T03:49:46.530000+00:00",
  "id": "/subscriptions/43f94930-ca31-4ddc-91c2-2f1be4f0e8f3/resourceGroups/cambridge/providers/Microsoft.ServiceBus/namespaces/cambridgena",
  "location": "China East 2",
  "metricId": "43f94930-ca31-4ddc-91c2-2f1be4f0e8f3:cambridgena",
  "name": "cambridgena",
  "provisioningState": "Succeeded",
  "resourceGroup": "cambridge",
  "serviceBusEndpoint": "https://cambridgena.servicebus.chinacloudapi.cn:443/",
  "sku": {
    "capacity": null,
    "name": "Standard",
    "tier": "Standard"
  },
  "tags": {},
  "type": "Microsoft.ServiceBus/Namespaces",
  "updatedAt": "2019-07-18T03:57:15.180000+00:00"
}


Create a service bus queue. Set the lock duration to 5 minutes. This means that the lock for each queue message will last for 5 minutes.

In [40]:
!az servicebus queue create \
    --resource-group {resource_group} \
    --namespace-name {namespace} \
    --name {queue} \
    --lock-duration PT5M

{
  "accessedAt": "0001-01-01T00:00:00",
  "autoDeleteOnIdle": "10675199 days, 2:48:05.477581",
  "countDetails": {
    "activeMessageCount": 0,
    "deadLetterMessageCount": 0,
    "scheduledMessageCount": 0,
    "transferDeadLetterMessageCount": 0,
    "transferMessageCount": 0
  },
  "createdAt": "0001-01-01T00:00:00",
  "deadLetteringOnMessageExpiration": false,
  "defaultMessageTimeToLive": "10675199 days, 2:48:05.477581",
  "duplicateDetectionHistoryTimeWindow": "0:10:00",
  "enableBatchedOperations": true,
  "enableExpress": false,
  "enablePartitioning": false,
  "forwardDeadLetteredMessagesTo": null,
  "forwardTo": null,
  "id": "/subscriptions/43f94930-ca31-4ddc-91c2-2f1be4f0e8f3/resourceGroups/cambridge/providers/Microsoft.ServiceBus/namespaces/cambridgena/queues/cambridgequ",
  "location": "China East 2",
  "lockDuration": "0:05:00",
  "maxDeliveryCount": 10,
  "maxSizeInMegabytes": 5120,
  "messageCount": 0,
  "name": "cambridgequ",
  "requiresDuplicateDetection": false,
 

Now that we've created our Service Bus namespace and queue, we need to get the key-name/key-value pair so that we can access it.

By default, your Service Bus resource will come with an key-value "authorization rule" pair - its key name will have the value: "RootManageSharedAccessKey". The following command will get the key name for the "authorization rule", and assign it as `sb_key_name`.

In [41]:
sb_key_name = !az servicebus namespace authorization-rule list \
    --resource-group {resource_group} \
    --namespace-name {namespace} \
    -o json --query "[0].name"

sb_key_name = str(sb_key_name[0][1:-1])

Get the primary key value to "RootManageSharedAccessKey".

In [42]:
sb_credentials = !az servicebus namespace authorization-rule keys list \
    --resource-group {resource_group} \
    --namespace-name {namespace} \
    --name {sb_key_name} \
    -o json --query "primaryKey"

In [43]:
sb_key_value = re.findall(r'"(.*?)"', str(sb_credentials))[0]

Set the service bus key value to the dotenv file.

In [44]:
set_key(env_path, "SB_SHARED_ACCESS_KEY_VALUE", sb_key_value) # generated
set_key(env_path, "SB_SHARED_ACCESS_KEY_NAME", sb_key_name) # generated

(True, 'SB_SHARED_ACCESS_KEY_NAME', 'RootManageSharedAccessKey')

---

### Set environment variables to be used by later notebooks

Check that our `.env` file looks correct.

In [45]:
!cat .env

SUBSCRIPTION_ID="43f94930-ca31-4ddc-91c2-2f1be4f0e8f3"
RESOURCE_GROUP="cambridge"
REGION="chinaeast2"
STORAGE_ACCOUNT_NAME="cambridgesa"
STORAGE_CONTAINER_NAME="cambridgesc"
STORAGE_MODEL_DIR="models"
SB_NAMESPACE="cambridgena"
SB_QUEUE="cambridgequ"
DOCKER_LOGIN="pjh177787"
SCORING_IMAGE="cambridgesp"
FLASK_IMAGE="cambridgefp"
AKS_CLUSTER="cambridgecl"
LOGIC_APP="cambridgela"
STORAGE_ACCOUNT_KEY="8JkanaXYAxmVdXoh9+Hbyr5EqTv5S4g+6ikHWt6JMZ2ivtSNKKWpjVy3vZALcSK4mQVRW3S+lVgoL6xFEHEIDw=="
SB_SHARED_ACCESS_KEY_VALUE="GPAQrJb0ZQ64kjo4IxIbdVClyPFTEZuji4jusi5oboc="
SB_SHARED_ACCESS_KEY_NAME="RootManageSharedAccessKey"


### Set up Azure Blob as a File System on our local machine using Blobfuse

Install blob fuse

In [46]:
ubuntu_version = !lsb_release -r | grep -o '[0-9][0-9].[0-9][0-9]'

In [47]:
# !wget https://packages.microsoft.com/config/ubuntu/{ubuntu_version[0]}/packages-microsoft-prod.deb
# !sudo dpkg -i packages-microsoft-prod.deb
# !sudo apt-get update

In [48]:
# !sudo apt-get install blobfuse

Setup ramdisk tmp dir for blob fuse for low latency buffer.

In [49]:
!sudo rm -rf /mnt/ramdisk && sudo mkdir /mnt/ramdisk
!sudo mount -t tmpfs -o size=4g tmpfs /mnt/ramdisk
!sudo mkdir /mnt/ramdisk/blobfusetmp
!sudo chown $(whoami) /mnt/ramdisk/blobfusetmp

rm: cannot remove '/mnt/ramdisk': Device or resource busy


Setup credentials for blob connection

In [50]:
!rm -f fuse_connection.cfg
!touch fuse_connection.cfg
!echo accountName {get_key(env_path, "STORAGE_ACCOUNT_NAME")} >> fuse_connection.cfg
!echo accountKey {get_key(env_path, "STORAGE_ACCOUNT_KEY")} >> fuse_connection.cfg
!echo containerName {get_key(env_path, "STORAGE_CONTAINER_NAME")} >> fuse_connection.cfg
!echo blobEndpoint {get_key(env_path, "STORAGE_ACCOUNT_NAME")}.blob.core.chinacloudapi.cn >> fuse_connection.cfg
!chmod 700 fuse_connection.cfg

Create mount directory

In [51]:
!rm -rf data && mkdir data

Mount blob to mount directory

In [52]:
!blobfuse data \
    --tmp-path=/mnt/ramdisk/blobfusetmp  \
    --config-file=fuse_connection.cfg \
    -o attr_timeout=240 \
    -o entry_timeout=240 \
    -o negative_timeout=120

### Add Model Dir and Video to storage

Copy `models` dir to the `data` folder we just mounted.

In [53]:
!cp -r -n models data

Download an `orangutan.mp4` video to use throughout the tutorial.

In [54]:
!wget https://happypathspublic.blob.core.windows.net/videos/orangutan.mp4

--2019-07-18 03:58:36--  https://happypathspublic.blob.core.windows.net/videos/orangutan.mp4
Resolving happypathspublic.blob.core.windows.net (happypathspublic.blob.core.windows.net)... 52.239.214.164
Connecting to happypathspublic.blob.core.windows.net (happypathspublic.blob.core.windows.net)|52.239.214.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7961293 (7.6M) [video/mp4]
Saving to: ‘orangutan.mp4’

orangutan.mp4       100%[===================>]   7.59M  1.98MB/s    in 3.8s    

2019-07-18 03:58:41 (1.98 MB/s) - ‘orangutan.mp4’ saved [7961293/7961293]



Move that video into the mounted `data` folder.

In [55]:
!mv orangutan.mp4 data 

Check that our models dir and orangutan.mp4 is uploaded to our mounted blob storage container.

In [56]:
!ls data

models	orangutan.mp4


Continue to the next [notebook](/notebooks/02_local_testing.ipynb).